In [2]:
import tensorflow as tf
import numpy as np
import os
#import horovod.tensorflow as hvd


print(tf.__version__)

1.9.0


In [3]:
# TODO

# what is done:
# partially done resnet model in tensorflow

# what needs work:
# 1. Create summaries - need to figure out what metrics are needed. 
# print out images from each block
# testing accuracy vs step, training accuracy vs. step, loss vs step
# ROC, AUC

# 2. Input pipeline - lots of work! Best way is to .csv files of image directories and work with that

# 3. Loss & Accuracy calculations need to be reviewed after bringing in new input pipeline

# https://www.tensorflow.org/guide/datasets - figuring out how to import data
# https://www.tensorflow.org/guide/summaries_and_tensorboard - 

In [4]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3])
y_train=y_train.reshape(y_train.shape[0])

x_test=x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_train.shape[3])
y_test=y_test.reshape(y_test.shape[0])

num_class = 10

In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [75]:

def conv2D(layer, 
           ft_size, 
           name, 
           ksize=1, 
           strides=[1, 1, 1, 1], 
           padding="SAME", 
           initializer=tf.contrib.layers.xavier_initializer(),
           dtype=tf.float32
          ):
  
  with tf.variable_scope(name, reuse=tf.AUTO_REUSE):  
    w = tf.get_variable(name='w', shape=[ksize, ksize, layer.shape[3], ft_size], dtype=dtype, initializer=initializer)
    b = tf.get_variable(name='b', shape=[ft_size], dtype=dtype, initializer=tf.zeros_initializer())

    layer = tf.nn.conv2d(input=layer, filter=w, strides=strides, padding=padding)
    layer = tf.add(layer, b)

    return layer

def respath_fn(respath, in_layer):
    new_shape = [respath.get_shape().as_list()[0], np.amax([in_layer.get_shape().as_list()[1], respath.get_shape().as_list()[1]]), np.amax([in_layer.get_shape().as_list()[2], respath.get_shape().as_list()[2]]), np.amax([in_layer.get_shape().as_list()[3], respath.get_shape().as_list()[3]])]
    
    res_padding = [[0, 0], [0, new_shape[1]-respath.get_shape().as_list()[1]], [0, new_shape[2]-respath.get_shape().as_list()[2]], [0, new_shape[3]-respath.get_shape().as_list()[3]]]
    in_padding = [[0, 0], [0, new_shape[1]-in_layer.get_shape().as_list()[1]], [0, new_shape[2]-in_layer.get_shape().as_list()[2]], [0, new_shape[3]-in_layer.get_shape().as_list()[3]]]
    
    a = tf.add(tf.pad(in_layer, in_padding, 'CONSTANT'), tf.pad(respath, res_padding, 'CONSTANT'))
    
    return a
    
    

In [76]:
tf.reset_default_graph()
# graph

HEIGHT = x_train.shape[1]
WIDTH = x_train.shape[2]
CHANNELS = x_train.shape[3]
NUM_CLASS = num_class

x = tf.placeholder(dtype=tf.float32, shape=[None, WIDTH, HEIGHT, CHANNELS], name='x')
y = tf.placeholder(dtype=tf.int64, shape=None, name='y')
batch = tf.placeholder(dtype=tf.int64, shape=None, name='batch')

with tf.variable_scope("conv1", reuse=tf.AUTO_REUSE):
  layer = conv2D(layer=x, ft_size=64, name='_1', ksize=7, strides=[1, 2, 2, 1])

resnet_layer = layer

with tf.variable_scope("conv2", reuse=tf.AUTO_REUSE):
  layer = tf.nn.max_pool(value=layer, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='max_pool')
  for i in range(3):
    with tf.variable_scope("block_"+str(i), reuse=tf.AUTO_REUSE):
      layer = conv2D(layer, 64, '_0')
      layer = conv2D(layer, 64, '_1', ksize=3)
      layer = conv2D(layer, 256, '_2')

resnet_layer = respath_fn(resnet_layer, layer)
layer = resnet_layer

with tf.variable_scope("conv3", reuse=tf.AUTO_REUSE):      
  for i in range(4):
    with tf.variable_scope("block_"+str(i), reuse=tf.AUTO_REUSE):
      layer = conv2D(layer, 128, '_0')
      layer = conv2D(layer, 128, '_1', ksize=3)
      layer = conv2D(layer, 512, '_2')

resnet_layer = respath_fn(resnet_layer, layer)
layer = resnet_layer
    
with tf.variable_scope("conv4", reuse=tf.AUTO_REUSE): 
  for i in range(6):
    with tf.variable_scope("block_"+str(i), reuse=tf.AUTO_REUSE):
      layer = conv2D(layer, 256, '_0')
      layer = conv2D(layer, 256, '_1', ksize=3)
      layer = conv2D(layer, 1024, '_2')

resnet_layer = respath_fn(resnet_layer, layer)
layer = resnet_layer

with tf.variable_scope("conv5", reuse=tf.AUTO_REUSE): 
  for i in range(3):
    with tf.variable_scope("block_"+str(i), reuse=tf.AUTO_REUSE):
      layer = conv2D(layer, 512, '_0')
      layer = conv2D(layer, 512, '_1', ksize=3)
      layer = conv2D(layer, 2048, '_2')

with tf.variable_scope("output", reuse=tf.AUTO_REUSE):      
  layer = tf.nn.avg_pool(value=layer, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME', name='avg_pool')
#   layer = tf.contrib.layers.fully_connected(layer, 1000)
  layer = tf.contrib.layers.flatten(layer)
  w_loss = tf.get_variable(name='w', shape=[layer.shape[1], NUM_CLASS], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
  b_loss = tf.get_variable(name='b', shape=[NUM_CLASS], dtype=tf.float32, initializer=tf.zeros_initializer())
  layer = tf.matmul(layer, w_loss) + b_loss

with tf.variable_scope("loss", reuse=tf.AUTO_REUSE):
  global_step = tf.train.get_or_create_global_step()
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(y, 10), logits=layer))
  optimize = tf.train.AdamOptimizer(learning_rate=1e-5).minimize(loss, global_step=global_step)
 # optimize = hvd.DistributedOptimizer(optimize)

with tf.variable_scope("accuracy", reuse=tf.AUTO_REUSE):     
  softmax = tf.argmax(tf.nn.softmax(logits=layer, axis=1), axis=1)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(softmax, y), tf.float32))*100

saver = tf.train.Saver()  
    
print(layer)

Tensor("output/add:0", shape=(?, 10), dtype=float32)


In [78]:

config = tf.ConfigProto()
# config.intra_op_parallelism_threads =68
# config.inter_op_parallelism_threads =4
# tf.session(config=config)
# this code supposedly needs to be on nersc

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True


with tf.Session(config=config) as sess:
  
    sess.run(tf.global_variables_initializer())
    
    for i in range(30):
        current_global_step, current_loss, current_accuracy, _ = sess.run([global_step, loss, accuracy, optimize], feed_dict={x: x_train[0:50], y: y_train[0:50]})
        
        if i % 5 == 0:
            current_accuracy = sess.run([accuracy], feed_dict={x: x_test[0:10], y: y_test[0:10]})
            print('loss: ', current_loss)
            print('accuracy: %.2f%%' % current_accuracy[0])

    current_accuracy = sess.run([accuracy], feed_dict={x: x_train[0:50], y: y_train[0:50]})
    print('accuracy: %.2f%%' % current_accuracy[0])            

    summary_writer = tf.summary.FileWriter('./tmp', graph=sess.graph)
    summary_op = tf.summary.merge_all() 
    
    saver.save(sess, save_path='./tmp/model.chkpt', global_step=current_global_step)

KeyboardInterrupt: 

In [33]:
d = np.ones([16, 32, 32, 64])
z = np.ones([16, 28, 28, 128])

np.amax([d.shape[1], z.shape[1]])

a = np.zeros([d.shape[0], np.amax([d.shape[1], z.shape[1]]), np.amax([d.shape[2], z.shape[2]]), np.amax([d.shape[3], z.shape[3]])])
b = np.zeros(a.shape)
print(b.shape)

a[:d.shape[0], :d.shape[1], :d.shape[2], :d.shape[3]] = d
b[:z.shape[0], :z.shape[1], :z.shape[2], :z.shape[3]] = z

ret = np.add(a, b).shape


SyntaxError: invalid syntax (<ipython-input-33-851075204988>, line 6)